<h1 style="color: #aadafa; text-align: center; font-size: 48px; margin-bottom: 0.2em;">
PW 1 - Introduction to SAR data and interferometric processing
<h1>
<p style="text-align: center; font-size: 18px; margin-top: 0.5em;">
<strong> Marie Pizzini</strong> 
</p>
<p style="text-align: center; font-size: 18px; margin-top: 0.5em;">
Telecom Paris - May 2024
</p>

The aim of this PW is to get familiar with SAR data (high dynamic of the images, complex data with ampltidue and phase). After a first part on the visualization of such data and the understanding of the image content, the second part is dedicated to the processing of phase and interferometric data.

## 1. Analysis of a series of SAR images
In this part, we will have a look at a temporal series of 25 SAR images acquired [from 2015-10-06 to 2016-11-05](https://perso.telecom-paristech.fr/tupin/TPSAR/pilelely/PileLely_3072x1024RECALZ4.label), with a time delay of approximately 12 days.

By making use of these SAR images, we will see the main differences with respect to conventional natural images and develop some simple tools to visualize an image, interpret it and detect temporal changes.

In [ ]:
!wget https://perso.telecom-paristech.fr/tupin/TPSAR/mvalab.py

import scipy
from scipy import signal
import scipy.signal
import scipy as spy
import scipy.fftpack
from scipy import ndimage
from scipy import special
from scipy import ndimage
import numpy as np
import math
import matplotlib.pyplot as plt
import mvalab as mvalab
from urllib.request import urlopen
import cmath
from google_drive_downloader import GoogleDriveDownloader as gdd


plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams['figure.max_open_warning'] = 30

In [ ]:
# Download of the time series
webpage = 'https://perso.telecom-paristech.fr/tupin/TPSAR/pilelely/multitemp/'
imagename = 'lely_tuple_multitemp.IMA'
imaslc = mvalab.imz2mat(webpage+imagename) # complex image

time_series_amp = np.abs(imaslc[0]) # taking an array of shape [n_rows,n_columns,n_dates] in amplitude format

### Question 1.1: display a SAR image
SAR images have a long-tailed distribution, making its dynamic range to be wide. Plot the image with matplotlib. What do you see?

It looks like an optical image with a low exposition and a low contrast. How can you improve the contrast of a SAR image, in order to have a more pleasant visualization?
Implement a function to plot a SAR image and comment the results obtained.

A usual way to do this, is the saturate all the values above a certain threshold (often chosen as the mean + 3 times the standard deviation of the image) and do a linear stretching for the other values.
Explain the effect of this processing and what you see in the resulting image.

### Answer 1.1
When plotting the image, we don't see anything else than a few small white dots which probably represent the points who have the highest intensity.

The dynamic range is too wide, we need to apply a threshold to only keep the interesting values which hold the main information of the image. The threshold is usually equal to the mean to which we add the standard deviation multplied by 3.

In [ ]:
plt.figure()
plt.imshow(time_series_amp[:,:,0],cmap='gray')
plt.show()

In [ ]:
# implement the following function to plot a SAR image
# you can use a threshold as mean + 3 sigma
# and the np.clip function

def plot_sar(ima):
  plt.figure(figsize=(12,12))
  t = ima.mean() + 3*ima.std() # choose the appropriate theshold
  plt.imshow(np.clip(ima,0,t),cmap='gray') # display the saturated image
  plt.show()

plot_sar(time_series_amp[:,:,0])

### Question 1.2: Image orientation
Is the image in the correct geographic orientation?
Apply the proper flipping if necessary.

To see the corresponding optical image and find the correct flipping, you can have a look on Google Maps at [this location](https://goo.gl/maps/LQcd3Uz9U7qCoRrH6).

### Answer 1.2
According to the orientation of the bridge, the image is flipped along the horizontal axis.

In [ ]:
plot_sar(time_series_amp[:,:,0])

In [ ]:
time_series_flipped = np.copy(time_series_amp)
time_series_flipped = np.flip(time_series_flipped,axis=0) # apply flipping if necessary using np.flip

In [ ]:
plot_sar(time_series_flipped[:,:,0])

### Question 1.3: Interpretation of a SAR image
The image contains
- an urban area,
- agricultural fields,
- water

Explain the appearence of these objects in the SAR image.

### Answer 1.3
- urban area: the urban areas are the brightest in the image. This is due to the materials used for construction which are really reflectant and reflect a lot of the signal back to the sensor.
The structure of the urban areas with a lot of walls creates more surface for the signal to bounce back to the sensor which also leads to brighter areas.

- agricultural fields: The agricultural fields are not as bright as the urban area and the brightnes is much more scattered. This can be due to the fact that the crops are not the same in each fields. Some might have higher plants than other which reflects better the signal while some might be more moist and absorb more the signal.

- water: the water areas are the darkest of the image which is due to the fact that water surface do not reflect the signal towards the sensor.
...

### Question 1.4: Multitemporal analysis and change detection
To detect changes occurring between three dates, images can be displayed in a false colour representation, each date being one channel of a RGB image. Implement the function ```plot_changes```.

Then, choose three dates from the multitemporal stack (indexes from 0 to 24). From the false colour representation, can you tell when a particular change has occurred? Explain how you can interpret it.

### Answer 1.4
The main differences (represented by color showing) are on the fields. This might be due to the fact that the crops have grown or have been harvested which results in a change of reflection.

On the other hand, the urban areas stay gray because the buildings didn't move during the different dates.

In [ ]:
def plot_changes(im1,im2,im3):
    threshold = im1.mean() + 3*im1.std() # select a unique threshold for the three images
    # apply thresholding and shrinking of each image between [0;1] for RGB data
    im1 = np.clip(im1,0,threshold)
    im1 = im1/threshold
    im2 = np.clip(im2,0,threshold)
    im2 = im2/threshold
    im3 = np.clip(im3,0,threshold)
    im3 = im3/threshold
    image = np.stack((im1,im2,im3), axis=-1) # create a false RGB image by stacking the three dates
    plt.figure(figsize=(12,12))
    plt.imshow(image)
    plt.show()

In [ ]:
# choose three dates
idx1 = 1
idx2 = 10
idx3 = 20
im1 = time_series_flipped[:,:,idx1]
im2 = time_series_flipped[:,:,idx2]
im3 = time_series_flipped[:,:,idx3]

plot_sar(im1)
plot_sar(im2)
plot_sar(im3)
plot_changes(im1,im2,im3)

### Question 1.5: Create an image with isotropic pixels
The image pixels have the following size: $13.86[m]\times 2.3[m]$ in azimuth $\times$ range (approximately, pixel height is 5 times greater than pixel length). How can you create an image with approximately squared pixels?

PS : the range drection was the horizontal direction and the azimuth direction the vertical direction in the original image (before any flipping)

### Answer 1.5
By convoluting the image with a mask that has the inverse ratio of the pixels, we can obtain squared pixel.

In [ ]:
ima_tmp = time_series_flipped[:,:,0]

mask = np.ones((1,5))/5
ima_filtered = scipy.signal.convolve2d(ima_tmp,mask) # filter the image with the appropriate mask
ima_squared_pixels = ima_filtered[:,::5] # resample the filtered image
plot_sar(ima_squared_pixels)

## 2. Analysis of TerraSAR-X images acquired over the Paris area
In this part we will use an image of TerraSAR-X sensor (metric resolution) of Paris.
This is a temporal stack, you can choose the date you want.
(The questions are similar to the previous ones and will not be done during the PW but later as personal work).

In [ ]:
# 21 images from TerraSAR-X between 2009-01-24 and 2010-11-26
webpage='https://perso.telecom-paristech.fr/tupin/TPSAR/paris/'
image='PileiEiffeljavelRECALZ4RECSP.IMA'
im_slc_tsx_paris_liste=mvalab.imz2mat(webpage+image);
im_slc_tsx_paris = im_slc_tsx_paris_liste[0][:,:,0]
plt.rcParams['figure.figsize'] = [18, 18]
mvalab.visusar(np.abs(im_slc_tsx_paris))

### Question 2.1: interpretation
Check that you recognize the main buildings on this image.
What is the position of the track of the sensor relatively to this image ?

Explain the appearence of the following buildings in the amplitude image : Eiffel Tower, Maison de la radio, Pont de Bir-Hakeim (you can use a [satellite optic image on googlemaps](https://www.google.com/maps/place/Eiffel+Tower/@48.851143,2.2797819,447m/data=!3m1!1e3!4m5!3m4!1s0x47e66e2964e34e2d:0x8ddca9ee380ef7e0!8m2!3d48.8583701!4d2.2944813) to help you).



### Answer 2.1
The sharp and reflective objects (like the eiffel tower) tend to lean towards the sensor which means the sensor was on the left of the image.

The differences between thos three buildings is due to the fact that they do not have the same architecture. The Eiffel Tower is made of metal and has a lot of edges whereas the Maison de la radio is hollow on the inside (we can only the sharp round edge)

### Question 2.2: change detection
Identify and explain the appearence of some changing objects on Paris.

### Answer 2.2
The region that changed correspond to small dots on the streets and rectangles on the Seine.

We can asume the dots correspond to cars whereas the rectangles might correspond to boats moving between the dates.

In [ ]:
idx1 = 1
idx2 = 10
idx3 = 20
im1 = np.abs(im_slc_tsx_paris_liste[0][:,:,idx1])
im2 = np.abs(im_slc_tsx_paris_liste[0][:,:,idx2])
im3 = np.abs(im_slc_tsx_paris_liste[0][:,:,idx3])

plot_sar(im1)
plot_sar(im2)
plot_sar(im3)
plot_changes(im1,im2,im3)

## 3. SAR interferometry
In this part, the objective is to see the content of the phase of a SAR image and to do a simple processing chain to obtain an interferogram with topographic fringes.
You will be able to have a look to the optical image [on this link](https://g.page/ParcoEtna?share)

In [ ]:
!wget https://perso.telecom-paristech.fr/tupin/TPSAR/interfero/Master.mat
!wget https://perso.telecom-paristech.fr/tupin/TPSAR/interfero/Slave.mat
imageM='./Master.mat'
imageS='./Slave.mat'
imamaitre=mvalab.matlab2imz(imageM,'Master')
imaslave=mvalab.matlab2imz(imageS,'Slave')

imamaitre = imamaitre[0][1000:3000,:]
imaslave = imaslave[0][1000:3000,:]
ncolonnes=imamaitre.shape[1]
nlignes=imamaitre.shape[0]

#%%
plt.rcParams['figure.figsize'] = [18, 18]
mvalab.visusar(imamaitre)
mvalab.visusar(imaslave)

In [ ]:
# display the phase of one image
#mvalab.visuinterfero uses an adapted lut to see the phase of an image
mvalab.visuinterfero(np.angle(imamaitre)+math.pi,0)


### Question 3.1
Do you recognize the area in the amplitude images ? Give a short interpretation. Visualize the phase of an image. Do you see any useful information?

### Answer 3.1

The image is a representation of the etan volcano.
We can see its crater on the left of the image.
On the right we can identify some changes in the topography which might represent some mountains.

When plotting the phase, we only get what could be discribed as noise which cannot be used usefully.

### Question 3.2
Compute the interferogram by computing $z_1.z_2^*$ and display the phase. Do you see any structured pattern ?

In [ ]:
raw_interf = imamaitre * np.conj(imaslave) # compute z1.z2*
plt.rcParams['figure.figsize'] = [18, 18]
mvalab.visuinterfero(np.angle(raw_interf)+math.pi,0) # diplay the phase

###Answer 3.2
We can see some vertical lines appearing. The deformation of the vertical lines seem to correspond to the one we see on the amplitude representation.

### Question 3.3: Registration of the two images
To compute an interferogram, the following steps have to be applied :
- registration of the two images (only a translation is searched)
- suppression of the vertical fringe pattern
- filtering to improve the phase information.

Using the knowledge acquired on previous courses, propose a method and apply it to do the registration between the two images. Complete the code below.

In [ ]:
# propose a method to extract the shift along the two axes
# to compute the correlation efficiently you can use the FFT

# using the multiplication in the Fourier domain, compute the FFT of the correlation and then the correlation by inverse FFT
imacorfft = np.conj(np.fft.fft2(np.abs(imamaitre))) * np.fft.fft2(np.abs(imaslave)) # to be completed
imacor= np.fft.ifft2(imacorfft) # to be completed

#visualize the correlation image (maximum gives the translation value)
mvalab.visusar(imacor)
ishift =  np.unravel_index(np.argmax(imacor), imacor.shape) # you can use np.argmax and np.unravel_index
print(ishift)

# compute the shift on the two axis
imaslaveroll=np.roll(imaslave, -ishift[0], axis=0)
imaslaveroll=np.roll(imaslaveroll, -ishift[1], axis=1)

# compute the interferogram and plot the interferometric phase
interfero= imamaitre * np.conj(imaslaveroll) # to be completed
plt.rcParams['figure.figsize'] = [18, 18]
mvalab.visuinterfero(np.angle(interfero)+math.pi,0)

### Question 3.4
What is the shift between the two images ? Which pattern do you see in the resulting fringes when computing the interferogram after the registration ?

### Answer 3.4
The shift between the two images is (2,0) which means the shift need to be done 2 pixels vertically.

We can see some clear parallel fringes on the image.

### Question 3.5: Suppression of the orbital fringes
To obtain the useful information about the topography the so-called orbital fringes have to be suppressed. It can be done by using the sensor parameters but in this practical work we propose to use a signal processing method.
How can you detect a pure sinusoid in a signal ? (it is the case of the pattern we would like to suppress in the horizontal direction). Compute the frequency of this pattern and use it to suppress it in the registered secondary image.

Which frequency did you obtain ? Is it the same for all the considered lines ? What do you see in the resulting interferogram after correction ?

In [ ]:
# compute the main frequency of the pattern by choosing a line in the image and using the FFT
frequencies=[]
max=0
min=np.inf
for i in range(interfero.shape[0]):
  fftfringe = np.fft.fft(interfero[i,:]) # to be completed
  frequence = np.unravel_index(np.argmax(np.abs(fftfringe)), fftfringe.shape)
  frequencies.append(frequence)
  if frequence[0]>max:
    max=i
  if frequence[0]<min:
    min=i

plt.figure()
plt.stem(np.fft.fft(interfero[max,:]))
print(np.argmax(np.abs(np.fft.fft(interfero[max,:]))))
plt.show()
plt.figure()
plt.stem(np.fft.fft(interfero[min,:]))
print(np.argmax(np.abs(np.fft.fft(interfero[min,:]))))
plt.show()


freq = np.mean(frequencies)
print(freq)

### Answer 3.5a
I decided to take the mean of the maximum frequency for each row.
We can see that the highest frequency ranges from 88 to 1968 depending on the row.

We get a mean frequency of 129.72

In [ ]:
N = interfero.shape[1]
n = np.arange(0,N)
onde = np.exp(1j*2*math.pi*freq*n/N)  # create a pure wave using the previously computed frequency freq
fonde = np.abs(np.fft.fft(onde))
plt.figure()
plt.stem(fonde)
plt.show()

In [ ]:
# suppress the pattern by multiplying the interferogram by the conjugate of the pure wave
interferofine= np.multiply(interfero,np.conj(onde))

mvalab.visuinterfero(np.angle(interferofine)+math.pi,0)

### Answer 3.5b
After correction, we see some lines that look closer to topographical lines. This is due to the fact that the orbital fringes were supressed.

Each line correspond to a certain level. The concetrical lines clearly show the topographic features of the crater.

### Question 3.6: interferogram filtering
To improve the quality of the interferogram, some filtering can be applied to reduce the phase noise. Propose a suitable kernel by filling the code below.

How does the filtered interferogram look like?

### Answer 3.6
...

In [ ]:
def interferogramme( tabvignette, tabvignette2, *therest) :
    """
    Computation of the complex interferogram between two images of the
    same size.
    By default, window size is set as 3x3 (dimx, dimy).
    It outputs a complex image, whose modulus is the coherence and whose phase
    is the interferometric phase.
    """

    dimx=3
    dimy=3

    if(len(therest)==1):
        dimx=therest[0]
        dimy=dimx

    if(len(therest)==2):
        dimx=therest[0]
        dimy=therest[1]

    nlig=np.size(tabvignette,0)
    ncol=np.size(tabvignette,1)

    if nlig != np.size(tabvignette2,0)  :
        print(u'les deux images doivent avoir la même taille (%d  et %d)'%(nlig, np.size(tabvignette2,0)))
        return 0

    if ncol != np.size(tabvignette2,1)  :
        print(u'les deux images doivent avoir la même taille')
        return 0

    interf= np.multiply(tabvignette, np.conj(tabvignette2))

    # part to be completed to compute a multi-look interfergram by averaging on a local window


    mask = np.ones((dimx,dimy))/(dimx*dimy) # to be completed
    interfiltr = signal.convolve2d(interf,mask,mode='same')

    den1 = np.sqrt(signal.convolve2d(np.square(np.abs(tabvignette)),mask,mode='same'))
    den2 = np.sqrt(signal.convolve2d(np.square(np.abs(tabvignette2)),mask,mode='same'))

    return interfiltr/(den1*den2+1e-12)

In [ ]:
imaslavefine= np.multiply(imaslaveroll,onde)
filtered_interferogram = interferogramme(imamaitre,imaslavefine)
mvalab.visusar(np.abs(filtered_interferogram))
mvalab.visuinterfero(np.angle(filtered_interferogram),0)

### Question 3.7
Can you apply the convolution directly on the interferometric phase image ?
What would happen in that case?

### Answer 3.7
The phase is 2pi-periodic. The periodicity could not be preserved by directly convoluting in the interferometric phase image. The dsicontinuities between -pi and pi can create some warping in the image.

